<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_TKE_PU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener==2.5.0
!pip install inquirer

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings
from scipy.signal import argrelextrema
import inquirer

warnings.simplefilter(action='ignore', category=FutureWarning)

# Kullanıcının tek bir zaman dilimi seçmesini sağlayan fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen zaman dilimini seçin:")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    choice = input("Zaman dilimini seçin (1-9): ")
    interval = intervals.get(choice, ('4 Saat', Interval.in_4_hour))[1]  # Varsayılan olarak 4 Saat
    return interval


# Teknik Göstergeler ve Fonksiyonlar
def rma(series, length=None):
    length = int(length) if length and length > 0 else 10
    alpha = (1.0 / length) if length > 0 else 0.5
    rma = series.ewm(alpha=alpha, min_periods=length).mean()
    return rma

def rsi(series, length=14):
    scalar = 100
    drift = 1
    negative = series.diff(drift)
    positive = negative.copy()
    positive[positive < 0] = 0
    negative[negative > 0] = 0
    positive_avg = rma(positive, length=length)
    negative_avg = rma(negative, length=length)
    rsi = scalar * positive_avg / (positive_avg + negative_avg.abs())
    return rsi

def cci(high, low, close, period=14, constant=0.015):
    typical_price = (high + low + close) / 3
    mean_deviation = typical_price.rolling(window=period).apply(lambda x: np.mean(np.abs(x - np.mean(x))))
    cci = (typical_price - typical_price.rolling(window=period).mean()) / (constant * mean_deviation)
    return cci

def williams_r(high, low, close, window=14):
    highest_high = high.rolling(window=window).max()
    lowest_low = low.rolling(window=window).min()
    williams_r = -100 * (highest_high - close) / (highest_high - lowest_low)
    return williams_r

def stochastic(high, low, close, length=14, smooth_k=3, smooth_d=3):
    highest_high = high.rolling(window=length).max()
    lowest_low = low.rolling(window=length).min()
    stoch = 100 * ((close - lowest_low) / (highest_high - lowest_low))
    return stoch

def mfi(high, low, close, volume, period=14):
    typical_price = (high + low + close) / 3
    raw_money_flow = typical_price * volume
    positive_flow = (raw_money_flow * (typical_price > typical_price.shift(1))).rolling(window=period).sum()
    negative_flow = (raw_money_flow * (typical_price < typical_price.shift(1))).rolling(window=period).sum()
    money_flow_ratio = positive_flow / negative_flow
    mfi = 100 - (100 / (1 + money_flow_ratio))
    return mfi

def momentum(close, window=14):
    momentum = close.diff(window)
    return momentum

def ultimate_oscillator(high, low, close, short_period=7, medium_period=14, long_period=28):
    true_low = pd.concat([low, close.shift(1)], axis=1).min(axis=1)
    true_high = pd.concat([high, close.shift(1)], axis=1).max(axis=1)
    bp = close - true_low
    true_range = true_high - true_low
    avg_bp_short = bp.rolling(window=short_period).sum()
    avg_tr_short = true_range.rolling(window=short_period).sum()
    avg_bp_medium = bp.rolling(window=medium_period).sum()
    avg_tr_medium = true_range.rolling(window=medium_period).sum()
    avg_bp_long = bp.rolling(window=long_period).sum()
    avg_tr_long = true_range.rolling(window=long_period).sum()
    ult_osc = 100 * ((4 * avg_bp_short / avg_tr_short) + (2 * avg_bp_medium / avg_tr_medium) + (avg_bp_long / avg_tr_long)) / (4 + 2 + 1)
    return ult_osc

def TKE(data, period=14, emaperiod=5, novolumedata=False):
    df = data.copy()
    df['Momentum'] = (df['close'] / df['close'].shift(period)) * 100
    df['CCI'] = cci(df['high'], df['low'], df['close'], period)
    df['RSI'] = rsi(df['close'], period)
    df['WILLR'] = williams_r(df['high'], df['low'], df['close'], period)
    df['STOCH'] = stochastic(df['high'], df['low'], df['close'], period)
    df['MFI'] = mfi(df['high'], df['low'], df['close'], df['volume'], period)
    df['Ultimate'] = ultimate_oscillator(df['high'], df['low'], df['close'], 7, 14, 28)
    if novolumedata:
        df['TKEline'] = (df['Ultimate'] + df['Momentum'] + df['CCI'] + df['RSI'] + df['WILLR'] + df['STOCH']) / 6
    else:
        df['TKEline'] = (df['Ultimate'] + df['MFI'] + df['Momentum'] + df['CCI'] + df['RSI'] + df['WILLR'] + df['STOCH']) / 7
    df['EMAline'] = df['TKEline'].ewm(span=emaperiod, adjust=False).mean()
    return df

def check_positive_divergence(data, price_col='close', tke_col='TKEline', lookback=10):
    recent_data = data[-lookback:]
    local_min_price = argrelextrema(recent_data[price_col].values, np.less_equal, order=2)[0]
    local_min_tke = argrelextrema(recent_data[tke_col].values, np.less_equal, order=2)[0]

    if len(local_min_price) < 2 or len(local_min_tke) < 2:
        return False

    recent_price_lows = recent_data[price_col].iloc[local_min_price[-2:]]
    recent_tke_lows = recent_data[tke_col].iloc[local_min_tke[-2:]]

    price_divergence = recent_price_lows.iloc[0] > recent_price_lows.iloc[1]
    tke_divergence = recent_tke_lows.iloc[0] < recent_tke_lows.iloc[1]

    return price_divergence and tke_divergence

# Ana kod
tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Kullanıcıdan zaman dilimini seçmesini istiyoruz
selected_interval = get_interval_choice()

Titles = ['Hisse Adı', 'Son Fiyat', 'TKE']
df_signals = pd.DataFrame(columns=Titles)

# Sembollerin döngüye alınması
for i in range(0, len(Hisseler)):
    try:
        # Seçilen zaman diliminde veriyi alıyoruz
        data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=selected_interval, n_bars=100)
        data = data.reset_index()
        data = TKE(data)
        latest_data = data.iloc[-1]
        if 10 < latest_data['TKEline'] < 25:
            if check_positive_divergence(data):
                L1 = [Hisseler[i], latest_data['close'], latest_data['TKEline']]
                df_signals.loc[len(df_signals)] = L1
                print(L1)
    except Exception as e:
        print(f"Error processing {Hisseler[i]}: {e}")

# Sonuçları yazdır
print(df_signals.to_string())



  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-tfyramgj
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-tfyramgj
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Lütfen zaman dilimini seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Zaman dilimini seçin (1-9): 7
['BRISA', 88.8, 23.631107586979518]
['ERSU', 17.77, 17.636588212982343]
['MEGAP', 2.94, 18.624722076079813]
['MEGMT', 31.58, 12.751362628683937]
['ORCAY', 9.23, 17.81065430536191]
['PETUN', 10.11, 13.052828496997478]
['PSDTC', 82.7, 12.720289075326681]
['SNGYO', 2.93, 19.210422415040203]
  Hisse Adı  Son Fiyat        TKE
0     BRISA      88.80  23.631108
1      ERSU      17.77  17.636588
2     MEGAP       2.94  18.624722
3     MEGMT      31.58  12.751363
4     ORCAY       9.23  17.810654
5     PETUN      10.11  13.052828
6     PSDTC      82.70  12.720289
7     SNGYO       2.93  19.210422
